In [1]:
import pickle
import os
import torch
import IPython.display as ipd
from util.evaluate import Evaluator
from factory.AutoVC_Adjust import AutoVC_Adjust

In [2]:
class Config:
    def __init__(self,root):
        self.root = root
        self.num_speaker = 40
        self.batch_size = 2
        self.erroment_uttr_idx = 16
        self.max_uttr_idx = 60
        self.len_crop = 176
        self.device = "cuda:0"
        self.all_speaker = sorted(next(iter(os.walk(root)))[1][:self.num_speaker])
        self.metadata = pickle.load(open(f'{root}/train.pkl', "rb"))[:self.num_speaker]

In [3]:
config = Config('train_data_vctk_ver2')

In [4]:
org_ad = AutoVC_Adjust(44,256,512,22).to(config.device)
org_ad.load_state_dict(torch.load("model/autovc_adjust_en.pt", map_location=config.device))

<All keys matched successfully>

In [5]:
E = Evaluator(config)

In [10]:
mel_source, mel_target, mel_trans = E.get_trans_mel(org_ad,11,25,isAdjust=True)

# Source

In [11]:
ipd.Audio(E.get_wavs(mel_source.transpose(2,1)),rate = 22050)

# Target

In [12]:
ipd.Audio(E.get_wavs(mel_target.transpose(2,1)),rate = 22050)

# Trans

In [13]:
ipd.Audio(E.get_wavs(mel_trans.transpose(2,1)),rate = 22050)